In [1]:
import config 
import Dataset
from Discriminator import Discriminator
from Generator import Generator
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import utils
import torch
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torchview import draw_graph
import albumentations as A
from albumentations.pytorch import ToTensorV2
import graphviz
import seaborn as sns
import os 
import cv2
from patchify import patchify, unpatchify
graphviz.set_jupyter_format('png')

'svg'

In [4]:
gan_checkpoint = "/home/gizmoo/Desktop/MA-code/Gen_L1/saves/save_06-02-2024_15:43:12/gen.pth.tar"
image = "/home/gizmoo/Desktop/BFACE/Masked_BFACE/Result of Blockface_RGB_66x66x60um_s841-s1083_234s0000.png"
trans = "/home/gizmoo/Desktop/BFACE/Masked_Transmittance/Result of Transmittance_66x66x60um_234s0000.png"
utils.make_GAN_prediction(image,trans,gan_checkpoint,config.CURVES_DIR)

=> Loading checkpoint
torch.Size([256, 256, 3])
torch.Size([256, 256, 3])
torch.Size([256, 256, 3])
torch.Size([256, 256, 3])


(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8),
 array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 